In [ ]:
!pip install numpy
!pip install pandas 
!pip install matplotlib

# Frequências Características de falha
**Frequência devido à falha local na pista externa ($f_e$):**
$$
f_e = \frac{n f_r}{2} \left( 1 - \frac{d}{D} \cos \phi \right)
$$

**Frequência devido à falha local na pista interna ($f_i$):**
$$
f_i = \frac{n f_r}{2} \left( 1 + \frac{d}{D} \cos \phi \right)
$$

**Frequência fundamental da carcaça ($f_C$):**
$$
f_C = \frac{f_r}{2} \left( 1 - \frac{d}{D} \cos \phi \right)
$$

**Frequência devido à falha na esfera ($f_B$):**
$$
f_B = \frac{D f_r}{2d} \left[ 1 - \left( \frac{d}{D} \cos \phi \right)^2 \right]
$$

# Amplitudes de Falha em função do diâmetro da falha

#### Falha na Esfera

1730 RPM Esfera (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :------------- | :-------------- | :----------------- |
| 0.007"         | 0.000076        | 68.906512          |
| 0.014"         | 0.000052        | 67.007271          |
| 0.021"         | 0.000061        | 67.203773          |


Diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.000052 | 67.007271 |
| 1750 | 0.000048 | 66.988548 |
| 1772 | 0.000038 | 70.937316 |
| 1797 | 0.000027 | 71.894030 |


#### Falha na Pista Interna

Falha na Pista Interna a 1730 RPM para diâmetros diferentes (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 0.007" | 0.005900 | 154.933817 |
| 0.014" | 0.008288 | 155.495846 |
| 0.021" | 0.011044 | 155.421302 |


Diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.008288 | 155.495846 |
| 1750 | 0.009055 | 157.674442 |
| 1772 | 0.009426 | 159.644141 |
| 1797 | 0.007026 | 161.712325 |

#### Pista Externa

Falha na Pista Externa a 1730 RPM para diâmetros diferentes (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 0.007" | 0.002440 | 103.384977 |
| 0.014" | 0.000360 | 103.286308 |
| 0.021" | 0.002292 | 102.991204 |


Falha na Pista Externa para diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.000360 | 103.286308 |
| 1750 | 0.000369 | 104.886496 |
| 1772 | 0.000420 | 106.307723 |
| 1797 | 0.000364 | 107.644075 |


In [16]:
import os
import pandas as pd
import numpy as np
import pprint

# =============================================================================
# BLOCO 1: CONFIGURAÇÃO, CARREGAMENTO E SEGMENTAÇÃO DOS DADOS
# =============================================================================

# --- 1. CONFIGURAÇÕES GERAIS ---

# INSTRUÇÃO: Altere esta linha para o caminho da sua pasta principal "DATASET".
caminho_raiz = r'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'

# Parâmetros dos rolamentos (Fonte: Case Western Reserve University)
params_drive_end = {'n': 9, 'd': 0.3126, 'D': 1.537, 'phi_graus': 0.0}
params_fan_end = {'n': 8, 'd': 0.2500, 'D': 1.122, 'phi_graus': 0.0}
TAXA_AMOSTRAL = 12000

# Dicionários para mapear informações dos nomes dos arquivos
mapa_tipo_falha = {'IR': 'Pista Interna', 'B': 'Esfera', 'OR': 'Pista Externa', 'Normal': 'Normal'}
mapa_diametro_falha = {'7': '0.007"', '14': '0.014"', '21': '0.021"'}
mapa_rpm_para_carga = {'1797': 0, '1772': 1, '1750': 2, '1730': 3}


# --- NOVA SEÇÃO: PARÂMETROS DE SEGMENTAÇÃO ---
tamanho_segmento = 4096
sobreposicao_percentual = 0.3  # 30% de overlap, conforme solicitado
passo = int(tamanho_segmento * (1 - sobreposicao_percentual)) # Hop size


# --- 2. CARREGAMENTO E PROCESSAMENTO DOS ARQUIVOS ---

dicionario_de_dataframes = {}
print(f"Iniciando a leitura e segmentação dos arquivos em '{caminho_raiz}'...")

for pasta_atual, _, arquivos in os.walk(caminho_raiz):
    for nome_arquivo in arquivos:
        if nome_arquivo.endswith('.npz'):
            caminho_completo = os.path.join(pasta_atual, nome_arquivo)
            
            # Decodificação de metadados do nome do arquivo (lógica original)
            nome_sem_ext = nome_arquivo.replace('.npz', '')
            partes = nome_sem_ext.split('_')
            rpm = partes[0]
            if 'Normal' in nome_arquivo:
                tipo_falha_cod, diametro_falha = 'Normal', 'N/A'
                sensores_para_processar = ['DE', 'FE'] 
            else:
                tipo_falha_cod = partes[1].split('@')[0]
                diametro_falha = mapa_diametro_falha.get(partes[2], 'Desconhecido')
                sensores_para_processar = ['DE'] if 'DE' in partes[-1] else ['FE']
            tipo_falha = mapa_tipo_falha.get(tipo_falha_cod, 'Desconhecido')

            try:
                dados_npz = np.load(caminho_completo)
                for sensor_cod in sensores_para_processar:
                    if sensor_cod in dados_npz.files:
                        # Carrega o sinal COMPLETO primeiro
                        sinal_completo = dados_npz[sensor_cod].ravel()
                        
                        # Define a chave base para o sinal (ex: '1730_B_14_DE')
                        if 'Normal' in nome_arquivo:
                            chave_base = f"{nome_sem_ext}_{sensor_cod}"
                        else:
                            partes_chave = nome_sem_ext.split('_')
                            partes_chave[-1] = partes_chave[-1].rstrip('0123456789')
                            chave_base = "_".join(partes_chave)
                            
                        # --- NOVO LOOP DE SEGMENTAÇÃO ---
                        for i, inicio in enumerate(range(0, len(sinal_completo) - tamanho_segmento + 1, passo)):
                            fim = inicio + tamanho_segmento
                            segmento = sinal_completo[inicio:fim]
                            
                            # Cria o DataFrame para este segmento
                            df_segmento = pd.DataFrame({'amplitude': segmento})
                            
                            # Adiciona todas as colunas de metadados
                            df_segmento['arquivo_origem'] = nome_arquivo
                            df_segmento['rotacao_rpm'] = int(rpm)
                            df_segmento['tipo_falha'] = tipo_falha
                            df_segmento['diametro_falha'] = diametro_falha
                            df_segmento['local_sensor'] = 'Drive End' if sensor_cod == 'DE' else 'Fan End'
                            
                            # Cria a chave única para o segmento e o adiciona ao dicionário
                            chave_segmento = f"{chave_base}_seg_{i}"
                            dicionario_de_dataframes[chave_segmento] = df_segmento

            except Exception as e:
                print(f"Erro ao processar o arquivo {nome_arquivo}: {e}")

if not dicionario_de_dataframes:
    print(f"\nERRO: Nenhum arquivo foi carregado. Verifique o caminho: '{caminho_raiz}'")
else:
    print(f"\nProcesso concluído! {len(dicionario_de_dataframes)} SEGMENTOS de dados foram carregados.")
    # Exibe uma chave de exemplo para verificação
    chave_exemplo = list(dicionario_de_dataframes.keys())[0]
    print(f"\nExemplo de um segmento (chave: '{chave_exemplo}'):")
    print(f"Tamanho do segmento: {len(dicionario_de_dataframes[chave_exemplo])} pontos")
    print(dicionario_de_dataframes[chave_exemplo].head())
    print(dicionario_de_dataframes.info())

Iniciando a leitura e segmentação dos arquivos em 'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'...

Processo concluído! 3958 SEGMENTOS de dados foram carregados.

Exemplo de um segmento (chave: '1730_B_14_DE_seg_0'):
Tamanho do segmento: 4096 pontos
   amplitude      arquivo_origem  rotacao_rpm tipo_falha diametro_falha  \
0   0.105420  1730_B_14_DE12.npz         1730     Esfera         0.014"   
1  -0.107370  1730_B_14_DE12.npz         1730     Esfera         0.014"   
2  -0.163410  1730_B_14_DE12.npz         1730     Esfera         0.014"   
3   0.118903  1730_B_14_DE12.npz         1730     Esfera         0.014"   
4   0.184039  1730_B_14_DE12.npz         1730     Esfera         0.014"   

  local_sensor  
0    Drive End  
1    Drive End  
2    Drive End  
3    Drive End  
4    Drive End  


AttributeError: 'dict' object has no attribute 'info'

In [14]:
# =============================================================================
# BLOCO 2: ANÁLISE DE FREQUÊNCIA E EXTRAÇÃO DE CARACTERÍSTICAS
# =============================================================================

# --- 1. FUNÇÕES DE ANÁLISE ---

def calcular_frequencias_rolamento(n, fr, d, D, phi_graus=0.0):
    """Calcula as frequências teóricas de falha de um rolamento."""
    phi_rad = np.deg2rad(phi_graus)
    termo_comum = (d / D) * np.cos(phi_rad)
    return {
        'Pista Externa': (n * fr / 2) * (1 - termo_comum),
        'Pista Interna': (n * fr / 2) * (1 + termo_comum),
        'Carcaça': (fr / 2) * (1 - termo_comum),
        'Esfera': (D * fr / (2 * d)) * (1 - termo_comum**2)
    }

def encontrar_amplitude_pico(sinal_temporal, taxa_amostral, freq_alvo, tolerancia_hz=3.0):
    """Encontra a amplitude de pico em um sinal na frequência alvo usando FFT."""
    N = len(sinal_temporal)
    if N == 0: return None, None
    
    yf = np.fft.fft(sinal_temporal)
    amplitude_espectral = 2.0/N * np.abs(yf[0:N//2])
    xf = np.fft.fftfreq(N, 1 / taxa_amostral)[:N//2]
    
    # Procura o pico na janela de frequência definida pela tolerância
    indices_janela = np.where((xf >= freq_alvo - tolerancia_hz) & (xf <= freq_alvo + tolerancia_hz))
    if len(indices_janela[0]) == 0: return None, None
    
    amplitude_pico = np.max(amplitude_espectral[indices_janela])
    indice_pico = np.argmax(amplitude_espectral[indices_janela])
    frequencia_pico = xf[indices_janela][indice_pico]
    
    return frequencia_pico, amplitude_pico

# --- 2. PROCESSAMENTO E EXTRAÇÃO ---

print("--- Iniciando análise de frequência dos sinais de falha ---")
resultados_analise = []

for chave, df in dicionario_de_dataframes.items():
    if df['tipo_falha'].iloc[0] != 'Normal':
        rpm = df['rotacao_rpm'].iloc[0]
        fr_hz = rpm / 60
        tipo_falha = df['tipo_falha'].iloc[0].strip()
        local_sensor = df['local_sensor'].iloc[0]
        sinal = df['amplitude'].values
        
        params_rolamento = params_drive_end if local_sensor == 'Drive End' else params_fan_end
        freqs_teoricas = calcular_frequencias_rolamento(fr=fr_hz, **params_rolamento)
        freq_alvo = freqs_teoricas.get(tipo_falha)
        
        if freq_alvo:
            freq_pico, amp_pico = encontrar_amplitude_pico(sinal, TAXA_AMOSTRAL, freq_alvo)
            if freq_pico is not None:
                resultados_analise.append({
                    'chave': chave, 'tipo_falha': tipo_falha, 'diametro_falha': df['diametro_falha'].iloc[0],
                    'rpm': rpm, 'local_sensor': local_sensor, 'freq_teorica_hz': freq_alvo,
                    'freq_encontrada_hz': freq_pico, 'amplitude_falha': amp_pico
                })

# --- 3. DATAFRAME FINAL DE RESULTADOS ---
df_resultados_geral = pd.DataFrame(resultados_analise)
df_resultados_de = df_resultados_geral[df_resultados_geral['local_sensor'] == 'Drive End'].copy()
df_resultados_fe = df_resultados_geral[df_resultados_geral['local_sensor'] == 'Fan End'].copy()

print(f"Análise concluída. {len(df_resultados_geral)} sinais de falha processados.")
print("\n--- Exemplo de Características Extraídas (Drive End) ---")
df_resultados_de.head(10)

--- Iniciando análise de frequência dos sinais de falha ---
Análise concluída. 67 sinais de falha processados.

--- Exemplo de Características Extraídas (Drive End) ---


,chave,tipo_falha,diametro_falha,rpm,local_sensor,freq_teorica_hz,freq_encontrada_hz,amplitude_falha
0,1730_B_14_DE,Esfera,"0.014""",1730,Drive End,67.952142,67.007271,0.000052
2,1730_B_21_DE,Esfera,"0.021""",1730,Drive End,67.952142,67.203773,0.000061
4,1730_B_7_DE,Esfera,"0.007""",1730,Drive End,67.952142,68.906512,0.000076
6,1730_IR_14_DE,Pista Interna,"0.014""",1730,Drive End,156.138972,155.495846,0.008288
8,1730_IR_21_DE,Pista Interna,"0.021""",1730,Drive End,156.138972,155.421302,0.011044
10,1730_IR_7_DE,Pista Interna,"0.007""",1730,Drive End,156.138972,154.933817,0.005900
12,1730_OR@6_14_DE,Pista Externa,"0.014""",1730,Drive End,103.361028,103.286308,0.000360
13,1730_OR@6_21_DE,Pista Externa,"0.021""",1730,Drive End,103.361028,102.991204,0.002292
14,1730_OR@6_7_DE,Pista Externa,"0.007""",1730,Drive End,103.361028,103.384977,0.002440
16,1750_B_14_DE,Esfera,"0.014""",1750,Drive End,68.737716,66.988548,0.000048


In [5]:
# =============================================================================
# BLOCO 3: GERAÇÃO DE DADOS SINTÉTICOS
# =============================================================================

# --- 1. PARÂMETROS DE GERAÇÃO ---

# ATENÇÃO: Estes valores são a base para a geração. Ajuste-os conforme a sua análise do Bloco 2.
amplitudes_referencia = {
    'Drive End': {'Esfera': 0.0001, 'Pista Interna': 0.001, 'Pista Externa': 0.0001},
    'Fan End': {'Esfera': 0.001, 'Pista Interna': 0.001, 'Pista Externa': 0.01}
}
multiplicadores = [0.001, 0.005, 0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
fases_para_adicionar_rad = [
    0, np.pi/4, np.pi/2, 3*np.pi/4,
    np.pi, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4
]

# --- 2. IDENTIFICAÇÃO DOS SINAIS NORMAIS ---

dicionario_sinais_normais = {
    chave: df for chave, df in dicionario_de_dataframes.items()
    if df['tipo_falha'].iloc[0] == 'Normal'
}
print(f"Encontrados {len(dicionario_sinais_normais)} sinais de operação normal para usar como base.")

# --- 3. GERAÇÃO E COMBINAÇÃO DOS SINAIS ---

lista_dados_finais = []
for chave_normal, df_normal in dicionario_sinais_normais.items():
    sinal_normal_base = df_normal['amplitude'].values
    N_PONTOS = len(sinal_normal_base)
    rpm_atual = df_normal['rotacao_rpm'].iloc[0]
    local_sensor = df_normal['local_sensor'].iloc[0]
    arquivo_origem = df_normal['arquivo_origem'].iloc[0]
    
    print(f"\nProcessando base: {arquivo_origem} ({rpm_atual} RPM, Sensor {local_sensor})")
    
    # Determina qual conjunto de parâmetros de rolamento usar
    params_rolamento = params_drive_end if local_sensor == 'Drive End' else params_fan_end
    
    # Calcula as frequências teóricas para a RPM atual
    fr_hz = rpm_atual / 60
    freqs_teoricas = calcular_frequencias_rolamento(fr=fr_hz, **params_rolamento)

    # Cria o vetor de tempo para a geração da onda senoidal
    duracao_s = N_PONTOS / TAXA_AMOSTRAL
    t = np.linspace(0.0, duracao_s, N_PONTOS, endpoint=False)
    
    for tipo_falha in ['Pista Externa', 'Pista Interna', 'Esfera']:
        freq_teorica = freqs_teoricas[tipo_falha]
        amp_ref = amplitudes_referencia[local_sensor][tipo_falha]
        
        for mult in multiplicadores:
            for fase in fases_para_adicionar_rad:
                amplitude_final = amp_ref * mult
                sinal_falha_sintetico = amplitude_final * np.sin(2 * np.pi * freq_teorica * t + fase)
                sinal_final_combinado = sinal_normal_base + sinal_falha_sintetico
                
                lista_dados_finais.append({
                    'sinal_final': sinal_final_combinado,
                    'tipo_falha_adicionada': tipo_falha,
                    'rpm': rpm_atual,
                    'multiplicador_amplitude': mult,
                    'fase_adicionada_rad': fase,
                    'arquivo_normal_base': arquivo_origem,
                    'local_sensor': local_sensor
                })

# --- 4. DATAFRAMES FINAIS COM DADOS SINTÉTICOS ---
df_final_geral = pd.DataFrame(lista_dados_finais)
df_final_de = df_final_geral[df_final_geral['local_sensor'] == 'Drive End'].reset_index(drop=True)
df_final_fe = df_final_geral[df_final_geral['local_sensor'] == 'Fan End'].reset_index(drop=True)

print("\n\n--- Geração de Dados Sintéticos Concluída! ---")
print(f"Total de {len(df_final_de)} sinais sintéticos gerados para o Drive End.")
print(f"Total de {len(df_final_fe)} sinais sintéticos gerados para o Fan End.")
print("\n--- Exemplo do DataFrame Final Gerado (Drive End) ---")
print(df_final_de.drop(columns=['sinal_final']).head())

Encontrados 8 sinais de operação normal para usar como base.

Processando base: 1730_Normal.npz (1730 RPM, Sensor Drive End)

Processando base: 1730_Normal.npz (1730 RPM, Sensor Fan End)

Processando base: 1750_Normal.npz (1750 RPM, Sensor Drive End)

Processando base: 1750_Normal.npz (1750 RPM, Sensor Fan End)

Processando base: 1772_Normal.npz (1772 RPM, Sensor Drive End)

Processando base: 1772_Normal.npz (1772 RPM, Sensor Fan End)

Processando base: 1797_Normal.npz (1797 RPM, Sensor Drive End)

Processando base: 1797_Normal.npz (1797 RPM, Sensor Fan End)


--- Geração de Dados Sintéticos Concluída! ---
Total de 1056 sinais sintéticos gerados para o Drive End.
Total de 1056 sinais sintéticos gerados para o Fan End.

--- Exemplo do DataFrame Final Gerado (Drive End) ---
  tipo_falha_adicionada   rpm  multiplicador_amplitude  fase_adicionada_rad  \
0         Pista Externa  1730                    0.001             0.000000   
1         Pista Externa  1730                    0.001     